In [3]:
#aysenur

In [4]:
import pandas as pd
import numpy as np

In [5]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger
import wandb

In [6]:
from TimeSeriesLearningUtils import get_data

In [7]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 currency_list,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage,
                 val_percentage,
                 test_percentage,
                 seq_len, 
                 ):
        self.currencies = currency_list
        self.n_currencies = len(self.currencies)
        self.x = torch.tensor(x[:self.n_currencies]).float()
        self.y = torch.tensor(y[:self.n_currencies]).long()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(self.n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(self.n_currencies)]
        
#         self.train_min = [self.x[i][:self.train_size].min() for i in range(n_currencies)]
#         self.train_max = [self.x[i][:self.train_size].max() for i in range(n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(self.n_currencies):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item[self.currencies[i] + "_window"] = window
            item[self.currencies[i] + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [8]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset,
                 val_dataset,
                 test_dataset,
                 calculate_loss_weights,
                 currencies,
                 num_classes,
                 window_size,
                 input_size,
                 batch_size,
                 lstm_hidden_sizes,
                 bidirectional,
                 learning_rate = 1e-3,
                 scheduler_step = 10,
                 scheduler_gamma = 0.1,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.currencies = currencies
        self.num_tasks = len(currencies)
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_sizes = lstm_hidden_sizes
        self.bidirectional = bidirectional 
        
        if calculate_loss_weights:
            loss_weights = []
            for i in range(self.num_tasks):
                train_labels = [int(train_dataset[n][self.currencies[i] +"_label"] )for n in range(train_dataset.__len__())]
                samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
                loss_weights.append((1 / samples_size) * sum(samples_size)/2)
            self.weights = loss_weights
        else:
            self.weights = None
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, 
                              num_layers=1, 
                              batch_first=True, 
                              hidden_size = self.lstm_hidden_sizes[0], 
                              bidirectional = bidirectional)
        self.batch_norm1 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[0])
        
        if len(self.lstm_hidden_sizes) > 1:
            self.lstm_2 = nn.LSTM(input_size = self.lstm_hidden_sizes[0], 
                                  num_layers=1, 
                                  batch_first=True, 
                                  hidden_size = self.lstm_hidden_sizes[1], 
                                  bidirectional = bidirectional)
            self.batch_norm2 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[1])

            self.lstm_3 = nn.LSTM(input_size = self.lstm_hidden_sizes[1], 
                                  num_layers=1, 
                                  batch_first=True, 
                                  hidden_size = self.lstm_hidden_sizes[2], 
                                  bidirectional = bidirectional)
            self.batch_norm3 = nn.BatchNorm2d(num_features=self.lstm_hidden_sizes[2])
        
        self.dropout = nn.Dropout(0.5)
        
        self.linear1 =[nn.Linear(self.lstm_hidden_sizes[-1], int(self.lstm_hidden_sizes[-1]/2))] * self.num_tasks
        self.linear1 = torch.nn.ModuleList(self.linear1)
        self.activation = nn.ReLU()
        
        self.output_layers = [nn.Linear(int(self.lstm_hidden_sizes[-1]/2), self.num_classes)] * self.num_tasks
        self.output_layers = torch.nn.ModuleList(self.output_layers)
        
        if self.weights != None:
            self.cross_entropy_loss = [nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) for weights in self.weights]
        else:
            self.cross_entropy_loss = [nn.CrossEntropyLoss() for _ in range(self.num_tasks)]
        
        self.cross_entropy_loss = torch.nn.ModuleList(self.cross_entropy_loss)
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
        self.learning_rate = learning_rate
        self.scheduler_step = scheduler_step
        self.scheduler_gamma = scheduler_gamma
        
    def forward(self, x, i):

        batch_size = x.size()[0]
        
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        
        x = self.dropout(x)

        x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm1(x.unsqueeze(0))
        
        if len(self.lstm_hidden_sizes) > 1:
            
            x = x.view(batch_size, self.window_size, x.size()[1])
            x, _  = self.lstm_2(x)

            x = self.dropout(x)

            x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
            x = self.batch_norm2(x.unsqueeze(0))

            x = x.view(batch_size, self.window_size, x.size()[1])
            x, _  = self.lstm_3(x)

            x = self.dropout(x)

            x = x.reshape(x.size()[-1], batch_size, self.window_size) #(feature_size, batch, window_len)
            x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, x.size()[1])
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
        x = self.dropout(x)
        
        x = self.linear1[i](x)
        x = self.activation(x)
                 
        output = self.output_layers[i](x)
    
        return output
    
    
    def training_step(self, batch, batch_nb):
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        # araştırılabilir
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self.forward(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_acc', acc, on_epoch=True, prog_bar=True)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_train_f1', f1, on_epoch=True, prog_bar=True)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        return loss 
    
    def validation_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
            #loss = F.nll_loss(output, y)
            loss += self.cross_entropy_loss[i](output, y)
 
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_acc', acc, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_val_f1', f1, on_epoch=True, prog_bar=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
    
    def test_step(self, batch, batch_nb):
        loss = torch.tensor(0.0, device="cuda:0") + torch.tensor(0.0, device="cuda:0")
        
        for i in range(self.num_tasks):
            x, y = batch[ self.currencies[i] + "_window"], batch[self.currencies[i] + "_label"]

            output = self(x, i)
#             print(y, torch.max(output, dim=1)[1])
#             print(F.softmax(output)) # mantıken fark etmiyor
            loss += self.cross_entropy_loss[i](output, y)
            
            acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_acc', acc, on_epoch=True, reduce_fx=torch.mean)

            f1 = self.f1_score(torch.max(output, dim=1)[1], y)
            self.log(self.currencies[i] +'_test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)

        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(model.parameters(), lr= self.learning_rate)#AdamW does weight decay
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                                    step_size=self.scheduler_step, 
                                                    gamma=self.scheduler_gamma)
        
        return [optimizer], [{"scheduler": scheduler}]
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [9]:
def name_model(config):
    task = "multi_task_" + "_".join(config["currency_list"]) if len(config["currency_list"]) > 1 else "single_task_" + config["currency_list"][0]
    classification = "multi_classification" if config["n_classes"] > 2 else "binary_classification"
    lstm = "stack_lstm" if len(config["lstm_hidden_sizes"]) > 1 else "single_lstm"
    trend_removed = "trend_removed" if config["remove_trend"] else ""
    bidirectional = "bidirectional" if config["bidirectional"] else ""
    loss_weighted = "loss_weighted" if config["loss_weight_calculate"] else ""

    return "_".join([task, bidirectional, lstm, loss_weighted, classification, trend_removed])

In [14]:
def experiment(config):
    MODEL_NAME = name_model(config)
    
    CURRENCY_LST = config["currency_list"]
    N_CLASSES = config["n_classes"]
    LSTM_HIDDEN_SIZES = config["lstm_hidden_sizes"]
    BIDIRECTIONAL = config["bidirectional"]
    REMOVE_TREND =config["remove_trend"]
    LOSS_WEIGHT_CALCULATE = config["loss_weight_calculate"]
    
    TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = config["dataset_percentages"] 
    WINDOW_SIZE = config["window_size"]
    FREQUENCY = config["frenquency"]
    NEUTRAL_QUANTILE = config["neutral_quantile"] if N_CLASSES > 2 else 0 
    BATCH_SIZE= config["batch_size"]
    #####
    
    X, y, features, dfs = get_data(CURRENCY_LST,
                                N_CLASSES,
                                 FREQUENCY, 
                                 WINDOW_SIZE,
                                 neutral_quantile = NEUTRAL_QUANTILE,
                                 log_price=True,
                                 remove_trend=REMOVE_TREND,
                                 include_indicators = False,
                                 include_imfs = False
                                )
    INPUT_FEATURE_SIZE = X.shape[-1]
    
    train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(CURRENCY_LST, 
                                                              X, 
                                                              y, 
                                                              dtype, 
                                                              TRAIN_PERCENTAGE, 
                                                              VAL_PERCENTAGE, 
                                                              TEST_PERCENTAGE, 
                                                              WINDOW_SIZE) for dtype in ['train', 'val', 'test']]
    
    config["dataset_sizes"] = [len(train_dataset), len(val_dataset), len(test_dataset)]
    ####
#     wandb.init(project="price_change",
#                config=config,
#                name = MODEL_NAME)
#     logger = WandbLogger()
    #logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_multi_task")

    model = LSTM_based_classification_model(
        train_dataset = train_dataset,
         val_dataset = val_dataset,
         test_dataset = test_dataset,
         calculate_loss_weights = LOSS_WEIGHT_CALCULATE,
         currencies = CURRENCY_LST,
         num_classes = N_CLASSES,
         window_size = WINDOW_SIZE,
         input_size = INPUT_FEATURE_SIZE,
         batch_size=BATCH_SIZE,
         lstm_hidden_sizes = LSTM_HIDDEN_SIZES,
         bidirectional = BIDIRECTIONAL)

    early_stop_callback = EarlyStopping(
       monitor='val_loss',
       min_delta=0.003,
       patience=10,
       verbose=True,
       mode='min'
    )

    trainer = pl.Trainer(gpus=-1, 
                         max_epochs= 150,
#                          logger = logger, 
                         callbacks=[early_stop_callback])
    trainer.fit(model)

    trainer.test()
    wandb.finish()

In [11]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16,
          "bidirectional": False}

In [12]:
CONFIG

{'window_size': 50,
 'dataset_percentages': [0.97, 0.007, 0.023],
 'frenquency': 'D',
 'neutral_quantile': 0.33,
 'batch_size': 16,
 'bidirectional': False}

In [61]:
from sklearn.model_selection import ParameterGrid

In [62]:
param_grid = {"n_classes": [2,3],
          "currency_list": [['BTC'], ['ETH'], ['LTC'], ['BTC', 'ETH'],  ['BTC', 'ETH', 'LTC']],
          "remove_trend": [True, False],
          "lstm_hidden_sizes": [[128], [128, 128, 128]],
          "loss_weight_calculate": [True, False]}

In [64]:
for c in (ParameterGrid(param_grid)):
    config = CONFIG.copy()
    config.update(c)
    print (config)
    experiment(config)

{'window_size': 50, 'dataset_percentages': [0.97, 0.007, 0.023], 'frenquency': 'D', 'neutral_quantile': 0.33, 'batch_size': 16, 'bidirectional': False, 'currency_list': ['BTC'], 'loss_weight_calculate': True, 'lstm_hidden_sizes': [128], 'n_classes': 2, 'remove_trend': True}


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type        | Params
---------------------------------------------------
0 | lstm_1             | LSTM        | 67.1 K
1 | batch_norm1        | BatchNorm2d | 256   
2 | dropout            | Dropout     | 0     
3 | linear1            | ModuleList  | 8.3 K 
4 | activation         | ReLU        | 0     
5 | output_layers      | ModuleList  | 130   
6 | cross_entropy_loss | ModuleList  | 0     
7 | f1_score           | F1          | 0     
8 | accuracy_score     | Accuracy    | 0     
---------------------------------------------------
75.7 K    Trainable params
0         Non-trainable params
75.7 K    Total params
0.303     Total estimated model params size (MB)
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a b

/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.3928571343421936,
 'BTC_test_f1': 0.18737061321735382,
 'ETH_test_acc': 0.2142857164144516,
 'ETH_test_f1': 0.11268097907304764,
 'LTC_test_acc': 0.2857142984867096,
 'LTC_test_f1': 0.1447124481201172,
 'test_loss': 1.115894079208374}
--------------------------------------------------------------------------------


BTC_train_acc_step,0.5625
BTC_train_f1_step,0.42222
ETH_train_acc_step,0.4375
ETH_train_f1_step,0.37465
LTC_train_acc_step,0.4375
LTC_train_f1_step,0.42121
train_loss_step,1.11167
epoch,10
trainer/global_step,803
_runtime,84
_timestamp,1621426279


BTC_train_acc_step,▃▃▅▄▅▄▅▅▅▆▄▇▁▂▅█
BTC_train_f1_step,▄▄▅▄▆▅▆▆▆▇▄█▁▁▅▇
ETH_train_acc_step,▂▃█▄▄▃▅▁▂▂▇▃▂▄▇▅
ETH_train_f1_step,▂▂█▄▃▃▅▁▂▂▆▃▂▃▆▄
LTC_train_acc_step,▂▆▆▅▇▅▃▆█▃▆▁█▅▇▇
LTC_train_f1_step,▂▆▄▃▆▄▃▆▆▃▅▁█▃▆▆
train_loss_step,█▇▂▄▄█▅▅▅▆▃▆▇▅▁▅
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████
trainer/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████


In [ ]:
for n in range(N_CURRENCIES):
    labels = [int(test_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(test_dataset.__len__())]
    print(pd.DataFrame({"label": labels}).groupby("label").size())

In [ ]:
for n in range(N_CURRENCIES):
    labels = [int(train_dataset[i]["currency_"+ str(n)+"_label"] )for i in range(train_dataset.__len__())]
    print(pd.DataFrame({"label": labels}).groupby("label").size())

In [ ]:
trainer.test(model, DataLoader(train_dataset, num_workers=15, pin_memory=True))

In [15]:
config = CONFIG.copy()
config.update({"n_classes": 2,
          "currency_list": ['BTC'],
          "remove_trend": True,
          "lstm_hidden_sizes": [128, 128, 128],
          "loss_weight_calculate": False})
experiment(config)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NameError: name 'model' is not defined

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback

def objective(trial):
    # PyTorch Lightning will try to restore model parameters from previous trials if checkpoint
    # filenames match. Therefore, the filenames for each trial must be made unique.


    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We create a simple logger instead that holds the log in memory so that the
    # final accuracy can be obtained after optimization. When using the default logger, the
    # final accuracy could be stored in an attribute of the `Trainer` instead.
    model = LSTM_based_classification_model()

    trainer = pl.Trainer(gpus=-1, 
                     max_epochs= 80,
                     logger = logger, 
                     callbacks=[PyTorchLightningPruningCallback(trial, monitor="loss")])

    trainer.fit(model)

    return logger.metrics[-1]["loss"]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
objective()